In [5]:
//input
val fileName1 = "dataset_abstracts/a9000006.txt"
val fileName2 = "dataset_abstracts/a9000031.txt"

val rows1 = sc.textFile(fileName1).map(line=>line.trim().replaceAll("(\\s)+", " ")).cache()
val text1 = rows1.reduce(_ + _)

val rows2 = sc.textFile(fileName2).map(line=>line.trim().replaceAll("(\\s)+", " ")).cache()
val text2 = rows2.reduce(_ + _)

fileName1 = dataset_abstracts/a9000006.txt
fileName2 = dataset_abstracts/a9000031.txt
rows1 = MapPartitionsRDD[23] at map at <console>:36
text1 = Commercial exploitation over the past two hundred years drovethe great Mysticete whales to near extinction. Variation inthe sizes of populations prior to exploitation, minimalpopulation size during exploitation and current populationsizes permit analyses of the effects of differing levels ofexploitation on species with different biogeographicaldistributions and life-history characteristics. Dr. StephenPalumbi at the University of Hawaii will study the geneticpopulation structure of three whale species in this context,the Humpback Whale, the Gray Whale and the Bowhead Whale. Theeffect of...


Commercial exploitation over the past two hundred years drovethe great Mysticete whales to near extinction. Variation inthe sizes of populations prior to exploitation, minimalpopulation size during exploitation and current populationsizes permit analyses of the effects of differing levels ofexploitation on species with different biogeographicaldistributions and life-history characteristics. Dr. StephenPalumbi at the University of Hawaii will study the geneticpopulation structure of three whale species in this context,the Humpback Whale, the Gray Whale and the Bowhead Whale. Theeffect of...

In [7]:
//k-gram
val k = 5
val k_gram1 = text1.split("").sliding(k).toList.map(x => (scala.util.hashing.MurmurHash3.arrayHash(x), ""))
val kgramRDD1 = sc.parallelize(k_gram1)
val filtered1 = kgramRDD1.distinct()

val k_gram2 = text2.split("").sliding(k).toList.map(x => (scala.util.hashing.MurmurHash3.arrayHash(x), ""))
val kgramRDD2 = sc.parallelize(k_gram2)
val filtered2 = kgramRDD2.distinct()




k = 5
k_gram1 = List((-368554769,""), (-2116654235,""), (1348093184,""), (-713942165,""), (154171951,""), (-1539835203,""), (621014717,""), (-702057327,""), (1061980120,""), (-1210501041,""), (-1777376094,""), (-2146599456,""), (1638764184,""), (2132630876,""), (-558803310,""), (-1704996955,""), (1554848983,""), (-643897612,""), (277879516,""), (1658643371,""), (-239693190,""), (-1096516904,""), (1369629706,""), (2053951917,""), (1404211726,""), (-496476476,""), (66848233,""), (-297738899,""), (1003038978,""), (229228928,""), (-120662840,""), (1068215916,""), (-475916955,""), (-443907787,""), (2075776880,""), (230583223,""), (1330534066,""), (-1120787669,""), (-1584074738,""), (-1702092513,""), (1354781654,""), (-1494390031,""), (-310258093,""), (1911875298,"")...


List((-368554769,""), (-2116654235,""), (1348093184,""), (-713942165,""), (154171951,""), (-1539835203,""), (621014717,""), (-702057327,""), (1061980120,""), (-1210501041,""), (-1777376094,""), (-2146599456,""), (1638764184,""), (2132630876,""), (-558803310,""), (-1704996955,""), (1554848983,""), (-643897612,""), (277879516,""), (1658643371,""), (-239693190,""), (-1096516904,""), (1369629706,""), (2053951917,""), (1404211726,""), (-496476476,""), (66848233,""), (-297738899,""), (1003038978,""), (229228928,""), (-120662840,""), (1068215916,""), (-475916955,""), (-443907787,""), (2075776880,""), (230583223,""), (1330534066,""), (-1120787669,""), (-1584074738,""), (-1702092513,""), (1354781654,""), (-1494390031,""), (-310258093,""), (1911875298,"")...

In [9]:

val join = filtered1.leftOuterJoin(filtered2)
val join_n = join.count().toFloat
val union = filtered1.union(filtered2).distinct()
val union_n = union.count().toFloat

val sim = join_n / union_n
val jacc_dist = 1-sim

println(sim)
println(jacc_dist)


0.51371205
0.48628795


join = MapPartitionsRDD[52] at leftOuterJoin at <console>:35
join_n = 1049.0
union = MapPartitionsRDD[56] at distinct at <console>:37
union_n = 2042.0
sim = 0.51371205
jacc_dist = 0.48628795


0.48628795

Now we work with Min-hashing

In [52]:
//Min-hashing

val p = 1073676287
val m = p + 1
def hashThis(a:Long, b:Long, x:Long): Long = {
    ((a*x + b) % p ) % m
}

//TODO: Outer loop: foreach document in set of documents
var doc_signature:Array[Long] = new Array[Long](100)
for(i<-0 to 99){
    
    val r = new scala.util.Random(i)
    val a2 = r.nextInt(p-1)
    val a = if(a2%2==0) a2+1 else a2
    val b = r.nextInt(p-1)
    
    val min = filtered1.map(s => s._1.toLong).reduce((x,y) => hashThis(a, b, x) min hashThis(a, b, y) )
    
    doc_signature(i) = min
    
}

p = 1073676287
m = 1073676288
doc_signature = Array(-342689503, -841956040, -99892218, -932095214, -577548700, -585859158, -619577524, -929598556, -903111229, -549820996, -849171872, -1020008119, -775718799, -998815566, -860483878, -600152099, -608684411, -505264634, -1026611494, -460002885, -901909936, -819056087, -819216546, -864331178, -725024484, -454154236, -296771822, -957145494, -798275923, -574231678, -1037749267, -1009271244, -975740647, -682134308, -770975105, -679068068, -1021970304, -239090677, -775895460, -431311941, -497539779, -412289549, -804483231, -385354347, -788001867, -1036339276, -541172357, -1054658366, -917418843, -686658522, -431932535, -799241785, -784821473, -371596209, -1013243213, -435818998, -...


hashThis: (a: Long, b: Long, x: Long)Long


Array(-342689503, -841956040, -99892218, -932095214, -577548700, -585859158, -619577524, -929598556, -903111229, -549820996, -849171872, -1020008119, -775718799, -998815566, -860483878, -600152099, -608684411, -505264634, -1026611494, -460002885, -901909936, -819056087, -819216546, -864331178, -725024484, -454154236, -296771822, -957145494, -798275923, -574231678, -1037749267, -1009271244, -975740647, -682134308, -770975105, -679068068, -1021970304, -239090677, -775895460, -431311941, -497539779, -412289549, -804483231, -385354347, -788001867, -1036339276, -541172357, -1054658366, -917418843, -686658522, -431932535, -799241785, -784821473, -371596209, -1013243213, -435818998, -...